# Import Libaries

In [79]:
import numpy as np
import pandas as pd 
import math
import seaborn as sns
import datetime as dt
from datetime import datetime    
sns.set_style("whitegrid")
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
from sklearn.model_selection import train_test_split

# Import Dataset

In [80]:
data = pd.read_csv('../../datasets/meta.csv')
data.head()

,Date,Open,High,Low,Close,Volume
0,2012-05-18,42.05,45.00,38.00,38.2318,573576400
1,2012-05-21,36.53,36.66,33.00,34.0300,168192700
2,2012-05-22,32.61,33.59,30.94,31.0000,101786600
3,2012-05-23,31.37,32.50,31.36,32.0000,73600000
4,2012-05-24,32.95,33.21,31.77,33.0300,50237200


# Data Preprocessing

In [81]:
# Reset index
data.reset_index(drop=True, inplace=True)

# Select numeric columns
numeric_data = data.select_dtypes(include=[np.number])

# Fill NaN values in numeric columns with the mean of those columns
data[numeric_data.columns] = data[numeric_data.columns].fillna(numeric_data.mean())

# Display the first few rows of the DataFrame
print(data.head())

         Date   Open   High    Low    Close     Volume
0  2012-05-18  42.05  45.00  38.00  38.2318  573576400
1  2012-05-21  36.53  36.66  33.00  34.0300  168192700
2  2012-05-22  32.61  33.59  30.94  31.0000  101786600
3  2012-05-23  31.37  32.50  31.36  32.0000   73600000
4  2012-05-24  32.95  33.21  31.77  33.0300   50237200


In [82]:
# Calculate moving average
ma_day = [5, 10, 20, 50]  # Common short-term and long-term windows
for ma in ma_day:
    column_name = f"MA for {ma} days"
    data[column_name] = data['Close'].rolling(window=ma).mean()

# Calculate Exponential moving average
for ma in ma_day:
    column_name = f"EMA for {ma} days"
    data[column_name] = data['Close'].ewm(span=ma, adjust=False).mean()

# Calculate RSI
def compute_RSI(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

data['RSI_14'] = compute_RSI(data)

# MACD
exp1 = data['Close'].ewm(span=12, adjust=False).mean()
exp2 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = exp1 - exp2




            Date    Open     High       Low     Close     Volume  \
0     2012-05-18   42.05   45.000   38.0000   38.2318  573576400   
1     2012-05-21   36.53   36.660   33.0000   34.0300  168192700   
2     2012-05-22   32.61   33.590   30.9400   31.0000  101786600   
3     2012-05-23   31.37   32.500   31.3600   32.0000   73600000   
4     2012-05-24   32.95   33.210   31.7700   33.0300   50237200   
...          ...     ...      ...       ...       ...        ...   
2668  2022-12-23  116.03  118.175  115.5350  118.0400   17796625   
2669  2022-12-27  117.93  118.600  116.0501  116.8800   21392311   
2670  2022-12-28  116.25  118.150  115.5100  115.6200   19612473   
2671  2022-12-29  116.40  121.030  115.7700  120.2600   22366192   
2672  2022-12-30  118.16  120.420  117.7400  120.3400   19583825   

      MA for 5 days  MA for 10 days  MA for 20 days  MA for 50 days  \
0               NaN             NaN             NaN             NaN   
1               NaN             NaN      

,count,mean,std,min,25%,50%,75%,max
Open,2673.0,1.479633e+02,8.696828e+01,1.808000e+01,7.780000e+01,1.422600e+02,1.913600e+02,3.816800e+02
High,2673.0,1.498468e+02,8.805465e+01,1.827000e+01,7.853000e+01,1.445600e+02,1.940300e+02,3.843300e+02
Low,2673.0,1.460862e+02,8.591529e+01,1.755000e+01,7.705000e+01,1.408700e+02,1.884700e+02,3.788100e+02
Close,2673.0,1.479917e+02,8.699929e+01,1.772900e+01,7.783000e+01,1.422800e+02,1.912900e+02,3.821800e+02
Volume,2673.0,3.115405e+07,2.721709e+07,5.913066e+06,1.618928e+07,2.313342e+07,3.599420e+07,5.735764e+08
MA for 5 days,2669.0,1.480981e+02,8.692966e+01,1.846140e+01,7.780600e+01,1.422620e+02,1.913780e+02,3.790300e+02
MA for 10 days,2664.0,1.482359e+02,8.686269e+01,1.877955e+01,7.777912e+01,1.418685e+02,1.917312e+02,3.786580e+02
MA for 20 days,2654.0,1.485193e+02,8.673551e+01,1.925918e+01,7.751544e+01,1.434165e+02,1.920813e+02,3.734220e+02
MA for 50 days,2624.0,1.493997e+02,8.630554e+01,2.017572e+01,7.745560e+01,1.471485e+02,1.906790e+02,3.637544e+02
EMA for 5 days,2673.0,1.479314e+02,8.694024e+01,1.846579e+01,7.757111e+01,1.420655e+02,1.913073e+02,3.782458e+02


In [83]:
# Plot the data
plt.plot(data['Close'], label="Close price")
plt.xlabel("Timestamp")
plt.ylabel("Closing price")
df = data
print(df)

df.describe().transpose()

In [84]:
# Remove any 'na' data
data.dropna(inplace=True)
data.head(500)

,Date,Open,High,Low,Close,Volume,MA for 5 days,MA for 10 days,MA for 20 days,MA for 50 days,EMA for 5 days,EMA for 10 days,EMA for 20 days,EMA for 50 days,RSI_14,MACD
49,2012-07-30,23.995,24.04,23.030,23.15,29285900,26.29800,27.52000,29.237800,29.838636,25.546291,27.046144,28.449746,30.538853,15.733114,-1.525536
50,2012-07-31,23.370,23.37,21.610,21.71,56179400,24.95000,26.88200,28.784750,29.508200,24.267527,26.075936,27.807865,30.192623,14.602446,-1.869091
51,2012-08-01,21.500,21.58,20.840,20.88,44604400,23.25800,26.05900,28.268750,29.245200,23.138351,25.131220,27.148068,29.827422,13.890909,-2.183168
52,2012-08-02,20.770,20.84,19.820,20.04,56374500,21.89700,25.16300,27.697250,29.026000,22.105568,24.205544,26.471110,29.443602,13.172414,-2.471369
53,2012-08-03,20.360,22.16,19.900,21.09,80647000,21.37400,24.39600,27.165250,28.807800,21.767045,23.639081,25.958623,29.116010,22.638623,-2.585243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,2014-07-21,68.810,69.96,68.500,69.40,49431600,67.81096,66.58973,66.626865,64.001716,68.007206,67.283914,66.478537,64.769416,56.251778,1.037495
545,2014-07-22,69.760,69.77,68.610,69.27,40301800,68.23196,67.24073,66.821865,64.242316,68.428138,67.645020,66.744391,64.945909,53.726471,1.134074
546,2014-07-23,69.740,71.33,69.610,71.29,77435900,68.95796,67.87273,67.100365,64.471516,69.382092,68.307744,67.177306,65.194697,64.538726,1.357958
547,2014-07-24,75.960,76.74,74.510,74.98,124006900,70.67198,68.88348,67.477365,64.774516,71.248061,69.520881,67.920420,65.578435,71.536342,1.812249


In [ ]:
# Adapted from the foundation code, but added own improvements

feature_columns = [ # Based on SHAP
    'Close',          # Close price of the day
    'Low',            # Low price of the day
    'High',           # High price of the day
    'Open',           # Open price of the day
    'EMA for 5 days', # Short-term trend indicator
    'MA for 5 days',  # Short-term trend indicator
    'RSI_14'          # Momentum indicator
]

X = []
Y = []
window_size = 15

# Iterating through the dataset to create inputs and targets
for i in range(1, len(df) - window_size - 1, 1):
    first_close = df.iloc[i, df.columns.get_loc('Close')]  # Normalize using "Close" price
    temp = []
    temp2 = []
    
    for j in range(window_size):
        # Normalize each feature in the feature_columns list
        normalized_features = []
        for feature in feature_columns:
            if feature == 'Close':
                # Normalize "Close" 
                normalized_value = (df.iloc[i + j, df.columns.get_loc(feature)] - first_close) / first_close
            else:
                normalized_value = df.iloc[i + j, df.columns.get_loc(feature)]
            normalized_features.append(normalized_value)
        
        # Append the normalized features 
        temp.append(normalized_features)
    
    # Calculate the target 
    target = (df.iloc[i + window_size, df.columns.get_loc('Close')] - first_close) / first_close
    temp2.append(target)
    
    # Append the input and target to X and Y
    X.append(np.array(temp))  
    Y.append(np.array(temp2).reshape(1, 1))

# Splitting the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

# Converting lists to numpy arrays
train_X = np.array(x_train)
test_X = np.array(x_test)
train_Y = np.array(y_train)
test_Y = np.array(y_test)

# Verify the shape of train_X
print("Shape of train_X before reshaping:", train_X.shape)

# Reshape the data for CNN models
train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1], train_X.shape[2])  
test_X = test_X.reshape(test_X.shape[0], 1, test_X.shape[1], test_X.shape[2])

# Print the number of training and testing samples
print("Shape of train_X after reshaping:", train_X.shape)
print("Shape of test_X after reshaping:", test_X.shape)
print("Number of training samples:", len(train_X))
print("Number of testing samples:", len(test_X))

Shape of train_X before reshaping: (2085, 15, 7)
Shape of train_X after reshaping: (2085, 1, 15, 7)
Shape of test_X after reshaping: (522, 1, 15, 7)
Number of training samples: 2085
Number of testing samples: 522


In [86]:
# Check if there are any NaN values

print(np.isnan(train_X).any(), np.isnan(train_Y).any())
print(np.isinf(train_X).any(), np.isinf(train_Y).any())

False False
False False


# Model Training

In [87]:
# Adapted from the foundation code, but added own improvements

import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.layers import MaxPooling1D, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


# Create the model
model = tf.keras.Sequential()

# Enhanced CNN layers with BatchNormalization and LeakyReLU
model.add(TimeDistributed(Conv1D(64, kernel_size=3, padding='same', 
                               kernel_regularizer=L1L2(l1=1e-5, l2=1e-4),
                               input_shape=(None, 100, 1))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU(alpha=0.1)))
model.add(TimeDistributed(MaxPooling1D(2)))

model.add(TimeDistributed(Conv1D(128, kernel_size=3, padding='same',
                               kernel_regularizer=L1L2(l1=1e-5, l2=1e-4))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU(alpha=0.1)))
model.add(TimeDistributed(MaxPooling1D(2)))

model.add(TimeDistributed(Conv1D(64, kernel_size=3, padding='same',
                               kernel_regularizer=L1L2(l1=1e-5, l2=1e-4))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU(alpha=0.1)))
model.add(TimeDistributed(MaxPooling1D(2)))

model.add(TimeDistributed(Flatten()))

# Enhanced LSTM layers 
model.add(Bidirectional(LSTM(128, return_sequences=True, 
                            kernel_regularizer=L1L2(l1=1e-5, l2=1e-4),
                            recurrent_regularizer=L1L2(l1=1e-5, l2=1e-4))))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Bidirectional(LSTM(64, return_sequences=True, 
                            kernel_regularizer=L1L2(l1=1e-5, l2=1e-4),
                            recurrent_regularizer=L1L2(l1=1e-5, l2=1e-4))))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Dense layers with regularization
model.add(Dense(32, kernel_regularizer=L1L2(l1=1e-5, l2=1e-4)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.2))

model.add(Dense(1, activation='linear'))

# Optimizer with gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)

model.compile(optimizer=optimizer,
             loss='mse',
             metrics=['mse', 'mae'])

# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6
    )
]

# Train the model
history = model.fit(
    train_X, 
    train_Y,
    validation_data=(test_X, test_Y),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

Epoch 1/100


c:\Users\timoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\timoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - loss: 1.0516 - mae: 0.6878 - mse: 0.7616 - val_loss: 0.2204 - val_mae: 0.0929 - val_mse: 0.0129 - learning_rate: 0.0010
Epoch 2/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.4960 - mae: 0.4248 - mse: 0.2959 - val_loss: 0.2092 - val_mae: 0.1192 - val_mse: 0.0185 - learning_rate: 0.0010
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.3605 - mae: 0.3234 - mse: 0.1706 - val_loss: 0.2103 - val_mae: 0.1330 - val_mse: 0.0224 - learning_rate: 0.0010
Epoch 4/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.3111 - mae: 0.2809 - mse: 0.1238 - val_loss: 0.1960 - val_mae: 0.0800 - val_mse: 0.0105 - learning_rate: 0.0010
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.2737 - mae: 0.2369 - mse: 0.0889 - val_loss: 0.1929 - val_mae: 0.0788 - val_mse: 0.0101 - learning_rate: 0.0010
Epoch 6/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.2450 - mae: 0.1939 - mse: 0.0629 - val_loss: 0.1883 - val_mae: 0.0674 - val_mse

Training Loss:

 [0.8272217512130737, 0.45216068625450134, 0.3425006866455078, 0.29803839325904846, 0.26642295718193054, 0.2404417097568512, 0.22377391159534454, 0.21073222160339355, 0.20129753649234772, 0.1913418173789978, 0.1828460693359375, 0.17747755348682404, 0.16914908587932587, 0.1631840020418167, 0.1578192561864853, 0.15191791951656342, 0.14686129987239838, 0.14146430790424347, 0.13488028943538666, 0.13015419244766235, 0.12440022826194763, 0.11923983693122864, 0.1143580824136734, 0.1091618463397026, 0.1045861765742302, 0.10003693401813507, 0.09510525315999985, 0.09036426246166229, 0.08635444939136505, 0.0816359594464302, 0.0779154822230339, 0.07365957647562027, 0.06931057572364807, 0.06576624512672424, 0.06219892203807831, 0.05858423188328743, 0.05530809983611107, 0.052041228860616684, 0.0492134690284729, 0.04642360657453537, 0.04369890317320824, 0.041519735008478165, 0.03840867802500725, 0.036497846245765686, 0.03449849784374237, 0.03222683072090149, 0.030215369537472725, 0.02824285812675953,

In [88]:
# plot train val loss 
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()

print("Training Loss:", history.history.get('loss', []))
print("Validation Loss:", history.history.get('val_loss', []))


In [89]:
# Evaluate the model
model.evaluate(test_X, test_Y)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0053 - mae: 0.0252 - mse: 0.0012    


[0.006304902024567127, 0.0021301417145878077, 0.03331639990210533]

In [90]:
# make predictions
predicted  = model.predict(test_X)
test_label = test_Y.reshape(-1,1)
predicted = np.array(predicted[:,0]).reshape(-1,1)
len_t = len(train_X)
for j in range(len_t , len_t + len(test_X)):
    temp = data.iloc[j,3]
    test_label[j - len_t] = test_label[j - len_t] * temp + temp
    predicted[j - len_t] = predicted[j - len_t] * temp + temp


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step


In [102]:
from scipy.signal import savgol_filter

def plot_predictions(test_label, predicted, title="Actual V Stock Price Prediction"):
    window_length = 5  # Must be odd
    polyorder = 1
    
    # Smooth both actual and predicted values
    test_label_smooth = savgol_filter(test_label.flatten(), window_length, polyorder)
    predicted_smooth = savgol_filter(predicted.flatten(), window_length, polyorder)
    
    plt.figure(figsize=(15, 6))
    
    # Plot smoothed lines
    plt.plot(test_label_smooth, label='Actual Price', color='red')
    plt.plot(predicted_smooth, label='Predicted Price', color='blue')
    
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.grid(True)
    plt.show()

In [103]:
plot_predictions(test_label, predicted, "Actual V Stock Price Prediction")

# Performance Evaluation

In [94]:
# Performance Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Calculate performance metrics
mse = mean_squared_error(test_label, predicted)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_label, predicted)
mape = np.mean(np.abs((test_label - predicted) / test_label)) * 100
r2 = r2_score(test_label, predicted)

# Print performance metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R^2 Score: {r2}")

Mean Squared Error (MSE): 105.08427391384267
Root Mean Squared Error (RMSE): 10.25106208711286
Mean Absolute Error (MAE): 7.67408290874247
Mean Absolute Percentage Error (MAPE): 3.54%
R^2 Score: 0.9845103629321943


In [95]:
# save the model
model.save('improved_cnn_lstm_model.keras')

In [98]:
def save_metrics(test_label, predicted, model_name, stock_name):
    """
    Calculate metrics and save to CSV 
    """
    # Calculate metrics
    mse = mean_squared_error(test_label, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_label, predicted)
    mape = np.mean(np.abs((test_label - predicted) / test_label)) * 100
    r2 = r2_score(test_label, predicted)
    
    # Round to 4 decimal places
    rmse = round(rmse, 4)
    mae = round(mae, 4)
    mape = round(mape, 4)
    r2 = round(r2, 4)
    mse = round(mse, 4)
    
    # Create metrics dictionary
    metrics = {
        'Model': [model_name],
        'Stock': [stock_name],
        'RMSE': [rmse],
        'MAE': [mae],
        'MAPE': [mape],
        'R2': [r2]
    }
    
    # Convert to DataFrame
    df_metrics = pd.DataFrame(metrics)
    
    # Save to CSV
    filename = f'metrics_{model_name}_{stock_name}.csv'
    df_metrics.to_csv(filename, index=False)
    
    # Print metrics
    print(f"\nMetrics for {model_name} on {stock_name}:")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}%")
    print(f"R² Score: {r2:.4f}")


In [99]:
# Save metrics to CSV
save_metrics(test_label, predicted, "CNN-LSTM-Improved", "META")  


Metrics for CNN-LSTM-Improved on META:
Mean Squared Error (MSE): 105.0843
Root Mean Squared Error (RMSE): 10.2511
Mean Absolute Error (MAE): 7.6741
Mean Absolute Percentage Error (MAPE): 3.5376%
R² Score: 0.9845


# Visualization & Model Architecture

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats

# Calculate residuals
residuals = test_label.flatten() - predicted.flatten()
plt.figure(figsize=(20, 12))

# 1. Residuals over time
plt.subplot(2, 2, 1)
plt.plot(residuals, 'b-', linewidth=1)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Residual (Actual - Predicted)')
plt.grid(True, alpha=0.3)

# 2. Histogram of residuals
plt.subplot(2, 2, 2)
sns.histplot(residuals, kde=True, bins=30)
plt.axvline(x=0, color='r', linestyle='--')
plt.title('Distribution of Residuals', fontsize=14, fontweight='bold')
plt.xlabel('Residual Value')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 3. Q-Q plot for normality assessment
plt.subplot(2, 2, 3)
stats.probplot(residuals, plot=plt)
plt.title('Q-Q Plot (Normality Check)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# 4. Residuals vs Predicted values
plt.subplot(2, 2, 4)
plt.scatter(predicted.flatten(), residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs Predicted Values', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Price')
plt.ylabel('Residual')
plt.grid(True, alpha=0.3)

# Summary 
stats_text = (
    f"Mean Residual: {np.mean(residuals):.4f}\n"
    f"Std Dev: {np.std(residuals):.4f}\n"
    f"RMSE: {np.sqrt(np.mean(residuals**2)):.4f}\n"
    f"MAE: {np.mean(np.abs(residuals)):.4f}"
)
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
plt.subplot(2, 2, 2)
plt.text(0.05, 0.95, stats_text, transform=plt.gca().transAxes, 
         fontsize=12, verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig('residual_analysis.png', dpi=300)
plt.show()

In [ ]:
# Plot the model architecture
print(model.summary())


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_26             │ (None, 1, 15, 64)      │         1,408 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_27             │ (None, 1, 15, 64)      │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_28             │ (None, 1, 15, 64)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_29             │ (None, 1, 7, 64)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_30             │ (None, 1, 7, 128)      │        24,704 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_31             │ (None, 1, 7, 128)      │           512 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_32             │ (None, 1, 7, 128)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_33             │ (None, 1, 3, 128)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_34             │ (None, 1, 3, 64)       │        24,640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_35             │ (None, 1, 3, 64)       │           256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_36             │ (None, 1, 3, 64)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_37             │ (None, 1, 1, 64)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_38             │ (None, 1, 64)          │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 1, 256)         │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 1, 128)         │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 1, 128)         │           51

 Total params: 1,256,069 (4.79 MB)

 Trainable params: 418,241 (1.60 MB)

 Non-trainable params: 1,344 (5.25 KB)

 Optimizer params: 836,484 (3.19 MB)

None
